In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

# 모델 선택 및 평가 관련 라이브러리
from sklearn.model_selection import train_test_split #,GridSearchCV(하이퍼파라미터), cross_val_score(교차 검증)
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report
                            #(accuracy는 맞춘 비율을 계산하여 정확도를 높이지만, 모델이 클래스 구분을 잘 했는지는 파악 불가
                            #roc는 확률값을 활용하여 모델의 성능 평가,, 클래스를 잘 구별하는 지 확인, 1에 가까울수록 좋은 모델
                                #accuracy는 데이터가 균형 잡혀 있을 때 G, roc는 데이터가 불균형할 때 G )

# 전처리: 범주형 인코딩, 결측치 대체, 스케일링
from sklearn.preprocessing import OrdinalEncoder       #범주형 데이터 숫자로 변환
from sklearn.preprocessing import StandardScaler       #데이터 표준화하여 평균 0, 표준편차 1로 변환, 데이터 크기 관련X 모델이 동일한 가중치 부여, (SVM, 로지스틱회귀, KNN, PCA 필수)
from sklearn.impute import SimpleImputer               #결측치 자동 대체 (strategy='mean':평균값, 'median':중앙값,'most_frequent':최빈값)


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/open/train.csv').drop(columns=['ID'])
test = pd.read_csv('/content/drive/MyDrive/open/test.csv').drop(columns=['ID'])

In [ ]:
# 타겟(입신 성공 여부) 분리 + 결측치 제거
y = train['임신 성공 여부']  # 타겟 변수 분리
x = train.drop(columns=['임신 성공 여부'])  # 입력 변수(x) 정의

# x에서 y의 결측치가 있는 행 제거
x = x.loc[y.dropna().index]
y = y.dropna()  # y에서도 결측치 제거


In [ ]:
#범주형 컬럼 전체 정의
categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부"
]

#범주형 컬럼을 문자열(str)로 변환
for col in categorical_columns:
  for col in categorical_columns:
    x.loc[:, col] = x[col].astype(str)
    test.loc[:, col] = test[col].astype(str)     #.loc[] 활용해야 안전

<ipython-input-5-e6a91ad6f264>:55: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1' '0' '1' ... '1' '1' '0']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  x.loc[:, col] = x[col].astype(str)
<ipython-input-5-e6a91ad6f264>:56: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1' '1' '1' ... '0' '0' '1']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  test.loc[:, col] = test[col].astype(str)     #.loc[] 활용해야 안전
<ipython-input-5-e6a91ad6f264>:55: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['0.0' '0.0' '0.0' ... '0.0' '0.0' '0.0']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  x.loc[:, col] = x[col].astype(str)
<ipython-input-5-e6a91ad6f264>:56: F

In [ ]:
#수치형 컬럼 전체 정의
numeric_columns = [
    "임신 시도 또는 마지막 임신 경과 연수",
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 채취 경과일",
    "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일",
    "배아 해동 경과일"
]

In [ ]:
#train 데이터를 train/val로 분할
x_train, x_val, y_train, y_val = train_test_split(
    x, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
# for col in categorical_columns:
#     most_frequent_value = x_train[col].mode()[0]  # 최빈값
#     x_train[col].fillna(most_frequent_value, inplace=True)
#     x_val[col].fillna(most_frequent_value, inplace=True)
#     test[col].fillna(most_frequent_value, inplace=True)

In [ ]:
# for col in categorical_columns:
#     freq = x_train[col].value_counts(normalize=True)  # 비율 계산
#     threshold = 0.01  # 1% 미만의 희귀값을 이상치로 간주
#     rare_values = freq[freq < threshold].index
#     x_train[col] = x_train[col].replace(rare_values, "Other")
#     x_val[col] = x_val[col].replace(rare_values, "Other")
#     test[col] = test[col].replace(rare_values, "Other")

In [ ]:
#범주형 변수 인코딩
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
x_train_enc = x_train.copy()
x_train_enc[categorical_columns] = ordinal_encoder.fit_transform(x_train_enc[categorical_columns])

x_val_enc   = x_val.copy()
x_val_enc[categorical_columns]   = ordinal_encoder.transform(x_val_enc[categorical_columns])


In [ ]:
# imputer = SimpleImputer(strategy='most_frequent')
# x_train_enc[numeric_columns] = imputer.fit_transform(x_train_enc[numeric_columns])
# x_val_enc[numeric_columns] = imputer.transform(x_val_enc[numeric_columns])

In [ ]:
#수치형 변수 결측치 대체 (중앙값 사용)
imputer = SimpleImputer(strategy='median')
x_train_enc[numeric_columns] = imputer.fit_transform(x_train_enc[numeric_columns])
x_val_enc[numeric_columns]   = imputer.transform(x_val_enc[numeric_columns])


In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.9 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostClassifier

# CatBoost 모델 설정
cat_model = CatBoostClassifier(
    iterations=200,
    learning_rate=0.1,
    depth=6,
    random_seed=42,
    loss_function='Logloss',
    verbose=10
)


In [ ]:

# 모델 학습 (early_stopping 적용)
cat_model.fit(
    x_train_enc, y_train,
    eval_set=[(x_val_enc, y_val)],
    early_stopping_rounds=10
)


0:	learn: 0.6298193	test: 0.6303025	best: 0.6303025 (0)	total: 206ms	remaining: 41s
10:	learn: 0.5041040	test: 0.5056392	best: 0.5056392 (10)	total: 1.87s	remaining: 32.1s
20:	learn: 0.4932851	test: 0.4952685	best: 0.4952685 (20)	total: 3.42s	remaining: 29.1s
30:	learn: 0.4903094	test: 0.4926833	best: 0.4926833 (30)	total: 5.33s	remaining: 29.1s
40:	learn: 0.4890647	test: 0.4918112	best: 0.4918112 (40)	total: 6.88s	remaining: 26.7s
50:	learn: 0.4881613	test: 0.4911550	best: 0.4911550 (50)	total: 8.88s	remaining: 25.9s
60:	learn: 0.4875254	test: 0.4907553	best: 0.4907553 (60)	total: 12.7s	remaining: 28.9s
70:	learn: 0.4870096	test: 0.4905007	best: 0.4905007 (70)	total: 15s	remaining: 27.2s
80:	learn: 0.4866276	test: 0.4903547	best: 0.4903547 (80)	total: 17.1s	remaining: 25.1s
90:	learn: 0.4862220	test: 0.4901770	best: 0.4901770 (90)	total: 19.1s	remaining: 22.8s
100:	learn: 0.4858430	test: 0.4900179	best: 0.4900158 (99)	total: 21.3s	remaining: 20.9s
110:	learn: 0.4854859	test: 0.4899263

In [ ]:
# 검증 데이터 ROC-AUC 평가
val_pred_proba = cat_model.predict_proba(x_val_enc)[:, 1]
val_roc_auc = roc_auc_score(y_val, val_pred_proba)
print("cat Validation ROC-AUC:", val_roc_auc)

cat Validation ROC-AUC: 0.7369089369691003


In [ ]:
#전체 학습 데이터 전처리 적용
x_full_enc = x.copy()
x_full_enc[categorical_columns] = ordinal_encoder.transform(x_full_enc[categorical_columns])  # fit X
x_full_enc[numeric_columns] = imputer.transform(x_full_enc[numeric_columns])  # fit X

In [ ]:
#최종 XGBoost 모델 (단, early stopping 없이 기본 n_estimators 사용)
model_full = CatBoostClassifier(
    iterations=200,
    learning_rate=0.1,
    depth=6,
    random_seed=42,
    loss_function='Logloss',
    verbose=10
)

model_full.fit(x_full_enc, y)

0:	learn: 0.6321419	total: 205ms	remaining: 40.8s
10:	learn: 0.5076008	total: 2.32s	remaining: 39.8s
20:	learn: 0.4936685	total: 5.64s	remaining: 48.1s
30:	learn: 0.4907926	total: 7.63s	remaining: 41.6s
40:	learn: 0.4894273	total: 8.9s	remaining: 34.5s
50:	learn: 0.4886859	total: 9.68s	remaining: 28.3s
60:	learn: 0.4881022	total: 10.4s	remaining: 23.8s
70:	learn: 0.4876173	total: 11.2s	remaining: 20.4s
80:	learn: 0.4871734	total: 12s	remaining: 17.6s
90:	learn: 0.4867937	total: 12.8s	remaining: 15.3s
100:	learn: 0.4864431	total: 13.6s	remaining: 13.3s
110:	learn: 0.4861238	total: 14.3s	remaining: 11.5s
120:	learn: 0.4858526	total: 15.1s	remaining: 9.85s
130:	learn: 0.4855429	total: 15.8s	remaining: 8.35s
140:	learn: 0.4851910	total: 17.2s	remaining: 7.2s
150:	learn: 0.4848864	total: 18.8s	remaining: 6.09s
160:	learn: 0.4846293	total: 20.3s	remaining: 4.91s
170:	learn: 0.4843650	total: 21s	remaining: 3.56s
180:	learn: 0.4840844	total: 21.7s	remaining: 2.28s
190:	learn: 0.4837982	total: 

**LightGBM**

In [ ]:
# from lightgbm import LGBMClassifier

# # LightGBM 모델 설정
# lgb_model = LGBMClassifier(
#     n_estimators=200,
#     learning_rate=0.1,
#     max_depth=6,
#     random_state=42,
#     metric='logloss'
# )


In [ ]:
# from lightgbm import LGBMClassifier, early_stopping, log_evaluation

# # 모델 학습 (early_stopping 적용)
# # 모델 학습 (early_stopping과 로그 출력 적용)
# lgb_model.fit(
#     x_train_enc, y_train,
#     eval_set=[(x_val_enc, y_val)],
#     eval_metric="logloss",  # 평가 지표
#     callbacks=[early_stopping(10), log_evaluation(1)]  # Early stopping 및 로그 콜백 사용
# )


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 52982, number of negative: 152098
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.110624 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 677
[LightGBM] [Info] Number of data points in the train set: 205080, number of used features: 62
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258348 -> initscore=-1.054573
[LightGBM] [Info] Start training from score -1.054573
[1]	valid_0's binary_logloss: 0.559848
Training until validation scores don't improve for 10 rounds
[2]	valid_0's binary_logloss: 0.550452
[3]	valid_0's binary_logloss: 0.542609
[4]	valid_0's binary_logloss: 0.535996
[5]	valid_0's binary_logloss: 0.530417
[6]	valid_0's binary_logl

LGBMClassifier(max_depth=6, metric='logloss', n_estimators=200, random_state=42)

In [ ]:
# # 검증 데이터 ROC-AUC 평가
# val_pred_proba = lgb_model.predict_proba(x_val_enc)[:, 1]
# val_roc_auc = roc_auc_score(y_val, val_pred_proba)
# print("LightGBM Validation ROC-AUC:", val_roc_auc)

LightGBM Validation ROC-AUC: 0.7365932419477756


In [ ]:
# #최종 XGBoost 모델 (단, early stopping 없이 기본 n_estimators 사용)
# model_full = LGBMClassifier(
#     n_estimators=100,
#     learning_rate=0.1,
#     max_depth=6,
#     random_state=42,
#     eval_metric='logloss'
# )

# model_full.fit(x_full_enc, y)

**XGBoost 모델 학습**

In [ ]:
# from xgboost import XGBClassifier

# # n_estimators를 크게 잡고 early_stopping_rounds를 통해 최적의 에포크에서 중단
# xgb_model = XGBClassifier(
#     n_estimators=200,
#     learning_rate=0.1,
#     max_depth=6,
#     random_state=42,
#     eval_metric='logloss'
# )

In [ ]:
# #fit 변수 eval_set, early stopping 사용
# xgb_model.fit(
#     x_train_enc, y_train,
#     eval_set=[(x_val_enc, y_val)],
#     #early_stopping_rounds=10,
#     verbose=True
# )

In [ ]:
# # 검증 데이터 ROC-AUC 평가
# val_pred_proba = xgb_model.predict_proba(x_val_enc)[:, 1]
# val_roc_auc = roc_auc_score(y_val, val_pred_proba)
# print("XGBoost Validation ROC-AUC:", val_roc_auc)

---

In [ ]:
#GridSearchCV를 활용한 하이퍼파라미터 튜닝 (마지막에 해보기. 디버깅문제)
"""
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [4, 6, 8],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}
grid_search = GridSearchCV(
    estimator=XGBClassifier(random_state=42, eval_metric='logloss'),
    param_grid=param_grid,
    scoring='roc_auc',
    cv=3,
    n_jobs=-1,
    verbose=1
)"""

"\nparam_grid = {\n    'n_estimators': [100, 200, 300],\n    'learning_rate': [0.01, 0.1, 0.2],\n    'max_depth': [4, 6, 8],\n    'subsample': [0.8, 1.0],\n    'colsample_bytree': [0.8, 1.0]\n}\ngrid_search = GridSearchCV(\n    estimator=XGBClassifier(random_state=42, eval_metric='logloss'),\n    param_grid=param_grid,\n    scoring='roc_auc',\n    cv=3,\n    n_jobs=-1,\n    verbose=1\n)"

In [ ]:
#전체 학습 데이터 전처리 적용
x_full_enc = x.copy()
x_full_enc[categorical_columns] = ordinal_encoder.transform(x_full_enc[categorical_columns])  # fit X
x_full_enc[numeric_columns] = imputer.transform(x_full_enc[numeric_columns])  # fit X

In [ ]:
# #최종 XGBoost 모델 (단, early stopping 없이 기본 n_estimators 사용)
# model_full = LGBMClassifier(
#     n_estimators=100,
#     learning_rate=0.1,
#     max_depth=6,
#     random_state=42,
#     eval_metric='logloss'
# )

# model_full.fit(x_full_enc, y)

[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Info] Number of positive: 66228, number of negative: 190123
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.130024 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 684
[LightGBM] [Info] Number of data points in the train set: 256351, number of used features: 62
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258349 -> initscore=-1.054568
[LightGBM] [Info] Start training from score -1.054568
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


LGBMClassifier(eval_metric='logloss', max_depth=6, random_state=42)

In [ ]:
# #최종 XGBoost 모델 (단, early stopping 없이 기본 n_estimators 사용)
# model_full = XGBClassifier(
#     n_estimators=100,
#     learning_rate=0.1,
#     max_depth=6,
#     random_state=42,
#     eval_metric='logloss'
# )
# model_full.fit(x_full_enc, y)

In [ ]:
#테스트 데이터 전처리 및 예측
x_test_enc = test.copy()
x_test_enc[categorical_columns] = ordinal_encoder.transform(x_test_enc[categorical_columns])
x_test_enc[numeric_columns] = imputer.transform(x_test_enc[numeric_columns])
test_pred_proba = model_full.predict_proba(x_test_enc)[:, 1]

In [ ]:
# 제출 파일 생성
sample_submission = pd.read_csv('/content/drive/MyDrive/open/sample_submission.csv')

sample_submission['probability'] = test_pred_proba
sample_submission.to_csv('./0222.3_cat.csv', index=False)
print("Finish! Submission file saved as 0217.01.csv")

Finish! Submission file saved as 0217.01.csv
